# Cvat auto-labeling for YoloV3 and YoloV4 

### In this notebook, it will show you how cvat auto-labeling for YoloV3 or YoloV4 works with AI-Maker inference. It depends on environment variable 'YOLO_VER' you passed to the inference to create YoloV3 or YoloV4 inference environment. You can reference templates we created for YoloV3 and YoloV4 inferences. We use YoloV3 as example in this notebook.

When the yolov3 inference creates, it will execute 'run.sh' at workdir "/opt/tensorrtserver".

In [ ]:
!cat run.sh

This script will first check inference environment (CPU or GPU) and export to ENV as later usage.

In [ ]:
%%bash
gpu_count=$(python3 check_env.py)
if [ $gpu_count > 0 ]
then
    export INFERENCE_DEVICE=GPU
else
    export INFERENCE_DEVICE=CPU
fi

echo $INFERENCE_DEVICE


Base on ENV "MODEL" to run corresponding part of script. In this case, MODEL is "YOLO".

In [ ]:
!echo $MODEL

Convert yolov3 weights from model management to onnx format according to environment variables defined in yolov3 template. You can check these environment variables. 

In [ ]:
!echo $CONFIG_FILE_PATH, $WEIGHTS_FILE_PATH, $LABEL_FILE_PATH, $YOLO_VER, $CONFIDENCE_THRESH, $NMS_THRESH

Run yolo_to_onnx.py to convert yolov3 weights to onnx format.

In [ ]:
!python3 yolo/yolo_onnx/yolo_to_onnx.py

Converted onnx model will be found at /models/yolov3/1

In [ ]:
!ls -al /models/yolov3/1


Set model_name from ENV "YOLO_VER".

In [ ]:
%%bash
if [ -v YOLO_VER ] && [ $YOLO_VER = 'V4' ]
then
    model_name="yolov4"
else
    model_name="yolov3"
fi

Get anchors defined in yolov3 config file and export to ENV as later usage.

In [ ]:
%%bash
anchors=$(python3 yolo/yolo_onnx/get_anchors.py)
export YOLO_ANCHORS=$anchors
echo $YOLO_ANCHORS

Create model config file by "build_config.py". See usage by argument "--help". 

In [ ]:
!python3 build_config.py --help

In [ ]:
!python3 build_config.py -m $model_name --max-batch-size 0 -f onnxruntime_onnx -d $INFERENCE_DEVICE

Model config file path is /models/{$model_name}/config.pbtxt.

In [ ]:
!cat /models/yolov3/config.pbtxt

We use Nvidia Triton inference server 19.10 as model inference engine, so we need to prepare model config file and put it at /models/yolov3 with model file as below directory structure. About Nvidia Triton inference server, you can reference [Triton Inference Server](https://github.com/triton-inference-server/server/tree/v1.7.0) for more informations.

In [ ]:
!ls -R /models/yolov3

Start triton inference server for model infernece, inference server will run in background and ready to accept inference requests by port 8000 for http and port 8001 for grpc.

In [ ]:
import subprocess
subprocess.Popen(["trtserver", "--model-repository=/models", "--strict-model-config=false"])

Check triton server status.

In [ ]:
!ps aux | grep trtserver

Check yolov3 model status.

In [ ]:
!curl http://localhost:8000/api/status/yolov3

Implement custom api for yolov3.
Two functions "cvat_invoke" and "cvat_info" of api.py need to be customize by user. This is yolov3 sample.

In [ ]:
!cat yolo/api.py

Function "cvat_invoke" will accept http post request from cvat server and return yolov3 inference results. Post data will be json format and be a base64 image for this yolov3 example.

In [ ]:
def cvat_invoke(post_json):
    url = "localhost:8001"
    protocol = "grpc"
    protocol = ProtocolType.from_str(protocol)
    if os.environ['YOLO_VER'] == 'V4':
        model_name = "yolov4"
    else:
        model_name = "yolov3"
    input_name, output_names, output_shapes, c, h, w = parse_model(url, protocol, model_name, 1, False)
    input_resolution_yolov3_HW = (h, w)
    preprocessor = PreprocessYOLO(input_resolution_yolov3_HW)
    image_raw, image = preprocessor.process(post_json["image"])
    shape_orig_WH = image_raw.size

    trt_outputs = []
    ctx = InferContext(url, protocol, model_name, 1, False, 0, False)
    input_batch = [image]

    output_dict = {
        output_name: InferContext.ResultFormat.RAW
        for output_name in output_names
    }

    response = ctx.run(
        {input_name: input_batch}, output_dict, len(input_batch))

    trt_outputs = [response[output][0] for output in sorted(response.keys())]
    trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, output_shapes)]
    anchors = get_anchors()
    if os.environ['YOLO_VER'] == 'V4':
        yolo_masks = [(0, 1, 2), (3, 4, 5), (6, 7, 8)]
    else:
        yolo_masks = [(6, 7, 8), (3, 4, 5), (0, 1, 2)]
    if os.environ.get('CONFIDENCE_THRESH'):
        obj_threshold = float(os.environ['CONFIDENCE_THRESH'])
    else:
        obj_threshold = 0.3
    if os.environ.get('NMS_THRESH'):
        nms_threshold = float(os.environ['NMS_THRESH'])
    else:
        nms_threshold = 0.5
    postprocessor_args = {"yolo_masks": yolo_masks,
                          "yolo_anchors": anchors,
                          "obj_threshold": obj_threshold,
                          "nms_threshold": nms_threshold,
                          "yolo_input_resolution": input_resolution_yolov3_HW}

    postprocessor = PostprocessYOLO(**postprocessor_args)

    boxes, classes, scores = postprocessor.process(trt_outputs, (shape_orig_WH))
    results = []
    if len(boxes) > 0 and len(classes) > 0 and len(scores) > 0:
        for box, score, category in zip(boxes, scores, classes):
            x_coord, y_coord, width, height = box
            left = int(max(0, np.floor(x_coord + 0.5).astype(int)))
            top = int(max(0, np.floor(y_coord + 0.5).astype(int)))
            right = int(min(image_raw.width, np.floor(x_coord + width + 0.5).astype(int)))
            bottom = int(min(image_raw.height, np.floor(y_coord + height + 0.5).astype(int)))
            results.append({"label": ALL_CATEGORIES[category], "points": [left, top, right, bottom], "type": "rectangle", "attributes": []})
    return results

Function "cvat_info" will return display informations for cvat server.

In [ ]:
def cvat_info():
    resp = {"framework": "yolov3", "spec": ALL_CATEGORIES, "type": "detector", "description": "Object detetion via Yolov3"}
    return resp

Install custom package if your customizations are ready.

In [ ]:
!python3 yolo/setup.py install

Run Flask http server with Waitress to serve inference request

In [ ]:
import subprocess
subprocess.Popen(["waitress-serve", "--call", "--port", "9999", "webapp:create_app"])

Check waitress status.

In [ ]:
!ps aux | grep waitress

Now your cvat auto-labeling for yolov3 at AI-Maker is ready to go.